# Reflectometry Analysis Example

We will start by working through a simple example of a neutron reflectometry analysis. 
This example doesn't have a specific life science application, but it is a good system to start with. 
We will use the Python library [`refnx`](https://refnx.readthedocs.io/en/latest/index.html) and this Jupyter Notebook to analyse our data. 

## Intended Learning Outcomes 

- Explain how `refnx` and Python can be used to analyse reflectometry data.
- Interpret the results of a reflectometry analysis. 
- Understand the use of optimisation and sampling in reflectometry. 

## Description of the data

The data you will investigate is from a polymer brush system; while not biological, it is an excellent demonstrative example. 
The data were collected at the Platypus reflectometer at the ANTSO reactor source in Australia. 
The data are stored as four columns ASCII files, with the columns representing the *q*-vectors, the measured reflectivity, the uncertainty in the measured reflectivity, and the width of a Gaussian resolution function. 
The specific system that we are investigating can be described with the following layers: 

```
|-----Silicon-----|
|-----------------|
|---Polymer Film--|
|-----------------|
|-------D2O-------|
```

We will look at a single neutron measurement. 

Let's begin with reading the data and having a look at it. 
We will use the `ReflectDataset` class from `refnx` to do this. 

In [ ]:
import matplotlib.pyplot as plt
from refnx.dataset import ReflectDataset

data = ReflectDataset('./data//simple.dat')

fig, ax = plt.subplots()
data.plot(fig=fig)
ax.set_xlabel('$q$ / Å')
ax.set_ylabel('$R(q)$')
ax.set_yscale('log')
plt.show()

## Building the Model

We have looked at the data to start constructing the model described above. 
The model will consist of four materials, the three above, plus a natural SiO<sub>2</sub> layer on the silicon block. 
We can construct these objects in Python as follows. 

In [ ]:
from refnx.reflect import SLD

si = SLD(2.07, name='Si')
sio2 = SLD(3.47, name='SiO2')
polymer = SLD(2.0, name='polymer')
d2o = SLD(6.36, name='d2o')

Notice that this function takes a single argument, the scattering length density of the material in units of 10 <sup>-6</sup> Å<sup>-2</sup>. 
Scattering length densities can have imaginary components, but for the materials investigated here, they are all 0. 

From these materials, we will now construct our layers; these are achieved by calling the material itself and passing an initial thickness and the roughness between this layer and the layer above. 
This is shown for the three layers below; no layer was created for the top layer, as this is semi-infinite. 

In [ ]:
sio2_layer = sio2(30, 3)
polymer_layer = polymer(250, 3)
d2o_layer = d2o(0, 3)

With the layers created, we can start to set the optimisation conditions, i.e., the parameters that can vary and the bounds within which they will vary. 
These parameters are set with the `setp` method, where we pass the bounds and the `vary=True` keyword arguments. 
Below, we define six parameters that will vary between the given bounds. 

In [ ]:
sio2_layer.thick.setp(bounds=(15, 50), vary=True)
sio2_layer.rough.setp(bounds=(1, 15), vary=True)

polymer_layer.thick.setp(bounds=(200, 300), vary=True)
polymer_layer.sld.real.setp(bounds=(0.1, 3), vary=True)
polymer_layer.rough.setp(bounds=(1, 15), vary=True)

d2o_layer.rough.setp(vary=True, bounds=(1, 15))

The next step is to construct the overall structure.
The other of these layers should match the expected structure of the material, with the first layer being the one that the neutron interacts with first. 

In [ ]:
structure = si | sio2_layer | polymer_layer | d2o_layer

We can plot the structure scattering length density profile as shown below.

In [ ]:
fig, ax = plt.subplots()
ax.plot(*structure.sld_profile())
ax.set_ylabel('SLD /$10^{-6} \AA^{-2}$')
ax.set_xlabel('distance / $\AA$')
plt.show()

Before we can perform the fitting, we need to add two more parameters to our model: the scale (the amount by which the calculated reflectometry should be scaled) and the background (a uniform background that is added to the data). 

In [ ]:
from refnx.reflect import ReflectModel

model = ReflectModel(structure, bkg=3e-6, dq=5.0)
model.scale.setp(bounds=(0.6, 1.2), vary=True)
model.bkg.setp(bounds=(1e-9, 9e-6), vary=True)

With the model wholly defined, it is possible to visualize a simulation of the model data over an example *q* range. 

In [ ]:
import numpy as np

q = np.linspace(0.005, 0.3, 1001)

fig, ax = plt.subplots()
ax.plot(q, model(q))
ax.set_xlabel('$q$ / Å')
ax.set_ylabel('$R(q)$')
ax.set_yscale('log')
plt.show()

## Fitting the Data

The model is now entirely constructed, so it is time to start the fitting. 
The fitting aims to modify the model parameters to get the best agreement between the model and the data. 
This is achieved with an `Objective` object. 

In [ ]:
from refnx.analysis import Objective

objective = Objective(model, data)

The optimisation is performed with an algorithm that refines our parameters, in this example we use the differential evolution algorithm which is extremely popular for reflectometry analysis. 

In [ ]:
from refnx.analysis import CurveFitter

fitter = CurveFitter(objective)
fitter.fit('differential_evolution')

When the optimisation is complete, we can plot our optimised model with our data, as shown below. 

In [ ]:
fig, ax = plt.subplots()
objective.plot(fig=fig)
ax.legend()
ax.set_xlabel('$q$ / Å')
ax.set_ylabel('$R(q)$')
ax.set_yscale('log')
plt.show()

We can also print the `objective`, which has all the parameter values (both being fitted and not). 

In [ ]:
print(objective)

## Performing Sampling

So far, we have only maximised the agreement between our model and the data, known as maximising the likelihood. 
However, it is becoming more and more popular to sample the full likelihood distribution. 
This gives us a statistical description of the values that may have been observed if our measurements were repeated over and over (assuming the uncertainties are correctly described in the measurement). 

We can sample this distribution using the `sample` method, as shown below. 

In [ ]:
fitter.sample(400)
fitter.reset()

We run the sampling for 400 steps and then reset this object (throwing away these samples). 
This is to allow the sampling system to reach some equilibrium before we perform the sampling that we will use in our analysis. 

In [ ]:
res = fitter.sample(15, nthin=100)

Above, we performed 1500 samples with 100 individual samplers. 
From each sampler, we use only every 100th sample (this is done to remove the correlation between the samples). 

Finally, it is possible to visualise the full probability distribution of our objective with the `corner` plot shown below.
The sampled distribution can also be visualised regarding the reflectivity and scattering length density. 

In [ ]:
objective.corner()
plt.show()

In [ ]:
objective.plot(samples=300)

In [ ]:
structure.plot(samples=300)
plt.ylim(2.2, 6)